In [48]:
import pandas as pd

In [49]:
from tqdm.auto import tqdm
tqdm.pandas()

In [50]:
pairs = pd.read_csv("../data/pairs.csv")
atoms = pd.read_csv("../data/atoms.csv")
stacked = pd.read_csv("../data/stacked.csv")

In [51]:
atoms[(atoms["file"] == "1d4r") & (atoms["nt"] == "B.A29")]

,file,nt,atom,x,y,z
7117347,1d4r,B.A29,P,15.958,-20.723,37.100
7117348,1d4r,B.A29,OP1,15.432,-22.024,37.596
7117349,1d4r,B.A29,OP2,15.177,-19.482,37.349
7117350,1d4r,B.A29,O5',17.422,-20.527,37.704


In [52]:
unique_nt_file = atoms[['nt', 'file']].drop_duplicates()

In [53]:
unique_nt_file

,nt,file
0,B.A1,2qk9
19,B.G2,2qk9
42,B.U3,2qk9
62,B.G4,2qk9
85,B.C5,2qk9
...,...,...
10588491,E.C17,6i1l
10588511,E.U18,6i1l
10588531,E.A19,6i1l
10588553,E.U20,6i1l


In [54]:
df = unique_nt_file

In [55]:
def generate_pairs_with_file(group):
    pairs = [f"{group.iloc[i]['nt']}, {group.iloc[i+1]['nt']}" for i in range(len(group) - 1)]
    files = group['file'].iloc[:-1].values  # Exclude the last value since we're creating pairs
    return pd.DataFrame({
        'file': files,
        'nt1': [pair.split(', ')[0] for pair in pairs],
        'nt2': [pair.split(', ')[1] for pair in pairs],
    })

# Apply the function to each group and concatenate the results
pairs_with_file_df = pd.concat([generate_pairs_with_file(group) for _, group in df.groupby('file')])

# Save the result to a new CSV file
#pairs_df.to_csv('/mnt/data/pairs.csv', index=False)

In [56]:
pairs_with_file_df[pairs_with_file_df["file"] == "1d4r"]

,file,nt1,nt2
0,1d4r,A.GDP1,A.G2
1,1d4r,A.G2,A.U3
2,1d4r,A.U3,A.G4
3,1d4r,A.G4,A.A5
4,1d4r,A.A5,A.C6
...,...,...,...
81,1d4r,C.C24,C.C25
82,1d4r,C.C25,C.A26
83,1d4r,C.A26,C.C27
84,1d4r,C.C27,C.U28


In [57]:
merged_df = pairs_with_file_df.merge(stacked, on=['file', 'nt1', 'nt2'], how='left', indicator=True)

In [58]:
merged_df['stacked'] = (merged_df['_merge'] == 'both').astype(int)

In [59]:
merged_df.drop('_merge', axis=1, inplace=True)

In [60]:
merged_df[merged_df["file"] == "1d4r"][:50]

,file,nt1,nt2,stacked
209,1d4r,A.GDP1,A.G2,0
210,1d4r,A.G2,A.U3,0
211,1d4r,A.U3,A.G4,0
212,1d4r,A.G4,A.A5,1
213,1d4r,A.A5,A.C6,1
214,1d4r,A.C6,A.C7,1
215,1d4r,A.C7,A.U8,0
216,1d4r,A.U8,A.C9,0
217,1d4r,A.C9,A.C10,0
218,1d4r,A.C10,A.C11,0


In [61]:
import pickle 
filehandler = open('matrix_nested_dict_names', 'rb') 
matrix_nested_dict_names = pickle.load(filehandler)

In [62]:
import numpy as np

In [63]:
def pairwise_distances(df, num_bins = 20):
    coords = df[['x', 'y', 'z']].values
    distances = np.sqrt(np.sum((coords[:, np.newaxis, :] - coords[np.newaxis, :, :]) ** 2, axis=-1))
    upper_triangle_indices = np.triu_indices_from(distances, k=1)
    flattened_distances = distances[upper_triangle_indices]
    histogram, bin_edges = np.histogram(flattened_distances, bins=num_bins, density=True)
    normalized_histogram = histogram / np.sum(histogram)
    return normalized_histogram, bin_edges

In [64]:
import itertools

In [65]:
def planar_angle(p1, p2, p3):
    b1 = p2 - p1
    b2 = p2 - p3

    angle = np.arccos(np.dot(b1, b2) / (np.linalg.norm(b1) * np.linalg.norm(b2)))
    return np.degrees(angle)


def torsion_angle(p1, p2, p3, p4):
    b1 = p2 - p1
    b2 = p3 - p2
    b3 = p4 - p3

    n1 = np.cross(b1, b2)
    n2 = np.cross(b2, b3)

    torsion = np.arctan2(
        np.dot(np.cross(n1, n2), b2 * np.linalg.norm(b2)), np.dot(n1, n2)
    )
    return np.degrees(torsion)

In [66]:
def gen_triplets_and_angle(vector1, vector2, num_bins=150):
    pairs_from_vector1 = list(itertools.combinations(vector1, 2))
    pairs_from_vector2 = list(itertools.combinations(vector2, 2))

    # Generate triplets where two elements are from vector1 and one from vector2
    triplets_1 = []
    for single_element in vector2:
        for pair in pairs_from_vector1:
            triplets_1.append((*pair, single_element))

    # Generate triplets where two elements are from vector2 and one from vector1
    triplets_2 = []
    for single_element in vector1:
        for pair in pairs_from_vector2:
            triplets_2.append((*pair, single_element))

    # Combine all triplets
    all_triplets = triplets_1 + triplets_2

    angles = []

    all_triplets = np.array(all_triplets)

    for trip in all_triplets:
        angles.append(planar_angle(trip[0], trip[1], trip[2]))
    
    histogram, bin_edges = np.histogram(angles, bins=num_bins, density=True)
    normalized_histogram = histogram / np.sum(histogram)
    return normalized_histogram, bin_edges

In [67]:
def gen_quad_and_angle(pos1, pos2, vector1, vector2, num_bins= 150):

    # Generate all possible pairs from each vector
    # pairs_from_vector1 = list(itertools.combinations(vector1, 2))
    # pairs_from_vector2 = list(itertools.combinations(vector2, 2))

    pairs_from_vector1 = [(vector1[pos1], atom) for atom in vector1 if list(atom) != list(vector1[pos1])]
    
    # Generate all pairs for pos2 with all other atoms in vector2 (excluding pos2)
    pairs_from_vector2 = [(vector2[pos2], atom) for atom in vector1 if list(atom) != list(vector2[pos2])]

    # zostawić C4' z pierwszego nukleotydu i wygenerować wszystkie pary z innymi 
    # tak samo zrobić dla drugiego nukleotydu 

    # Generate combinations of pairs

    combinations_of_pairs = []
    for pair1 in pairs_from_vector1:
        for pair2 in pairs_from_vector2:
            combinations_of_pairs.append(pair1 + pair2)

    
    combinations_of_pairs = np.array(combinations_of_pairs)

    # Print the combinations of pairs
    angles = [torsion_angle(quad[0], quad[1], quad[2], quad[3]) for quad in combinations_of_pairs]

    histogram, bin_edges = np.histogram(angles, bins=num_bins, density=True)
    normalized_histogram = histogram / np.sum(histogram)
    return normalized_histogram, bin_edges

In [68]:
def is_correct_according_to_rnaview(c2 = None, c6 = None, n1 = None):
    # Convert tuples to numpy arrays for vector operations
    c2 = np.array(c2)
    c6 = np.array(c6)
    n1 = np.array(n1)

    # Calculate distances using numpy's linear algebra norm function
    d1 = np.linalg.norm(c2 - c6)
    d2 = np.linalg.norm(n1 - c6)
    d3 = np.linalg.norm(n1 - c2)

#

    return (d1 <= 3.0 and
            d2 <= 2.0 and
            d3 <= 2.0)

In [69]:
clear_df = []

In [70]:
for i, (file, nt1, nt2, stacked) in merged_df.iterrows():
    if i % 100 == 0:
        print(i, "/", len(merged_df)-1, end='\r')
    try:
        df1 = matrix_nested_dict_names[(file, nt1)]
        df1 = np.array(df1)

        names_1 = df1[:, 0]
        c2_pos_1 = np.where(names_1 == "C2")[0][0]
        c6_pos_1  = np.where(names_1 == "C6")[0][0]
        n1_pos_1  = np.where(names_1 == "N1")[0][0]
        c2_1 = df1[c2_pos_1, 1:].astype(float)
        c6_1 = df1[c6_pos_1, 1:].astype(float)
        n1_1 = df1[n1_pos_1, 1:].astype(float)

        df1_ok = is_correct_according_to_rnaview(c2_1, c6_1, n1_1)

        df2 = matrix_nested_dict_names[(file, nt2)]
        df2 = np.array(df2)
        names_2 = df2[:, 0]
        c2_pos_2 = np.where(names_2 == "C2")[0][0]
        c6_pos_2 = np.where(names_2 == "C6")[0][0]
        n1_pos_2 = np.where(names_2 == "N1")[0][0]
        c2_2 = df2[c2_pos_2, 1:].astype(float)
        c6_2 = df2[c6_pos_2, 1:].astype(float)
        n1_2 = df2[n1_pos_2, 1:].astype(float)

        df2_ok = is_correct_according_to_rnaview(c2_2, c6_2, n1_2)

        if df1_ok and df2_ok:
            clear_df.append(True)
            #clear_df.append((file, nt1, nt2, stacked))
        else:
            clear_df.append(False)
    except:
        clear_df.append(False)
        #print((file, nt1, nt2, stacked))
        continue

In [73]:
merged_df_final = merged_df[clear_df]

In [74]:
merged_df_final

,file,nt1,nt2,stacked
0,1a9n,Q.C0,Q.C1,0
1,1a9n,Q.C1,Q.U2,0
2,1a9n,Q.U2,Q.G3,0
3,1a9n,Q.G3,Q.G4,0
4,1a9n,Q.G4,Q.U5,1
...,...,...,...,...
484909,8af0,D.G39,D.C40,0
484910,8af0,D.C40,D.G41,0
484911,8af0,D.G41,D.G42,0
484912,8af0,D.G42,D.G43,0


In [30]:
df = matrix_nested_dict_names[('7kfn', 'C.1W5/11')]
df = np.array(df)
names = df[:, 0]

In [31]:
names

array(['N', 'P', 'C1', 'C2', 'O2', 'N3', 'C4', 'N4', 'C5', 'C6', "C1'",
       "C2'", "C3'", "O3'", "C4'", "O4'", "C5'", "O5'", 'ON1', 'ON2',
       'OP1', 'OP2'], dtype='<U32')

In [32]:
df

array([['N', '-59.146', '24.05', '26.123'],
       ['P', '-63.735', '22.291', '28.291'],
       ['C1', '-58.932', '20.404', '26.386'],
       ['C2', '-57.584', '20.254', '25.809'],
       ['O2', '-57.043', '19.138', '25.69'],
       ['N3', '-56.905', '21.315', '25.396'],
       ['C4', '-57.383', '22.534', '25.487'],
       ['N4', '-56.64', '23.572', '25.061'],
       ['C5', '-58.706', '22.741', '26.042'],
       ['C6', '-59.456', '21.666', '26.493'],
       ["C1'", '-59.672', '19.22', '26.851'],
       ["C2'", '-59.538', '19.175', '28.37'],
       ["C3'", '-60.954', '18.993', '28.872'],
       ["O3'", '-61.081', '17.826', '29.671'],
       ["C4'", '-61.872', '18.864', '27.657'],
       ["O4'", '-61.064', '19.259', '26.522'],
       ["C5'", '-63.152', '19.702', '27.865'],
       ["O5'", '-62.738', '21.032', '28.18'],
       ['ON1', '-58.682', '25.001', '25.205'],
       ['ON2', '-59.894', '24.376', '27.037'],
       ['OP1', '-64.763', '21.939', '29.333'],
       ['OP2', '-63.014', '23.6

In [33]:
c2_1 = np.where(names == "C2")[0][0]
c2 = df[c2_1, 1:].astype(float)
c2

array([-57.584,  20.254,  25.809])

In [34]:
c6_1 = np.where(names == "C6")[0][0]
c6 = df[c6_1, 1:].astype(float)
c6

array([-59.456,  21.666,  26.493])

In [35]:
n1_1 = np.where(names == "N1")[0][0]
n1 = df[n1_1, 1:].astype(float)
n1

IndexError: index 0 is out of bounds for axis 0 with size 0

In [128]:
d1 = np.linalg.norm(c2 - c6)
print(d1)
d2 = np.linalg.norm(n1 - c6)
print(d2)
d3 = np.linalg.norm(n1 - c2)
print(d3)

2.4146902492866436
1.3879484860757625
1.388426447457695


In [80]:
lw_data = []

for i, (file, nt1, nt2, stacked) in merged_df_final.iterrows():
    if i % 100 == 0:
        print(i, "/", len(merged_df_final)-1, end='\r')


    df1 = matrix_nested_dict_names[(file, nt1)]
    df2 = matrix_nested_dict_names[(file, nt2)]

    df1 = np.array(df1)
    df2 = np.array(df2)


    try:
        names_1 = df1[:, 0]
        pos_1 = np.where(names_1 == "C4'")[0][0]
        names_2 = df2[:, 0]
        pos_2 = np.where(names_2 == "C4'")[0][0]
    except:
        print(i,file, nt1, nt2)
        print('names pos')
        continue


    df1 = np.array(df1[:,1:].astype(float))
    df2 = np.array(df2[:,1:].astype(float))

    try:
        planar_angles, _ = gen_triplets_and_angle(df1, df2, 50)
    except:
        print(i,file, nt1, nt2)
        print('planar_angles')
        continue

    try:
        torsion_angles, _ = gen_quad_and_angle(pos_1, pos_2, df1, df2, 50)
    except:
        print(i,file, nt1, nt2)
        print('torsion_angles')
        continue

    merged = np.concatenate((df1, df2))

    pairwise, _ = pairwise_distances(pd.DataFrame(merged, columns=['x','y','z']), num_bins=50)
    
    row = [*list(pairwise), *list(torsion_angles), *list(planar_angles), stacked] #*list(torsion_angles),
    lw_data.append(row)

C:\Users\Jurek\AppData\Local\Temp\ipykernel_31352\987503286.py:5: RuntimeWarning: invalid value encountered in scalar divide
  angle = np.arccos(np.dot(b1, b2) / (np.linalg.norm(b1) * np.linalg.norm(b2)))


75958 3p4c A.CFZ107 A.G108
planar_angles
75959 3p4c A.G108 B.CFZ209
planar_angles


C:\Users\Jurek\AppData\Local\Temp\ipykernel_31352\987503286.py:5: RuntimeWarning: invalid value encountered in scalar divide
  angle = np.arccos(np.dot(b1, b2) / (np.linalg.norm(b1) * np.linalg.norm(b2)))


433489 6ia2 A.A5 A.A6
planar_angles
433490 6ia2 A.A6 A.C7
planar_angles


In [81]:
col_names = [str(i) for i in range(150)]
col_names.append("stacked")

In [82]:
data_to_save = pd.DataFrame(lw_data, columns=col_names)
data_to_save.head()

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,stacked
0,0.022523,0.036036,0.001502,0.000000,0.000000,0.009009,0.066066,0.022523,0.013514,0.012012,...,0.005378,0.003866,0.004202,0.002185,0.002017,0.001849,0.001513,0.001008,0.000672,0
1,0.028205,0.026923,0.000000,0.000000,0.007692,0.065385,0.020513,0.010256,0.012821,0.021795,...,0.006579,0.003553,0.003553,0.003158,0.002500,0.002105,0.001974,0.001053,0.000921,0
2,0.035437,0.016611,0.000000,0.004430,0.060908,0.024363,0.008859,0.017719,0.026578,0.045404,...,0.004560,0.003075,0.003287,0.002121,0.001485,0.001166,0.001591,0.000318,0.000636,0
3,0.035749,0.013527,0.000000,0.009662,0.052174,0.021256,0.010628,0.016425,0.029952,0.052174,...,0.004812,0.003609,0.003867,0.003007,0.002320,0.000945,0.001117,0.000773,0.000430,0
4,0.031008,0.021041,0.000000,0.001107,0.016611,0.057586,0.014396,0.011074,0.017719,0.013289,...,0.006787,0.004242,0.002651,0.002863,0.002757,0.000530,0.001060,0.000848,0.000318,1


In [83]:
#data_to_save.to_csv('stacked_full.csv',",", index=None)

In [84]:
data_to_save

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,stacked
0,0.022523,0.036036,0.001502,0.000000,0.000000,0.009009,0.066066,0.022523,0.013514,0.012012,...,0.005378,0.003866,0.004202,0.002185,0.002017,0.001849,0.001513,0.001008,0.000672,0
1,0.028205,0.026923,0.000000,0.000000,0.007692,0.065385,0.020513,0.010256,0.012821,0.021795,...,0.006579,0.003553,0.003553,0.003158,0.002500,0.002105,0.001974,0.001053,0.000921,0
2,0.035437,0.016611,0.000000,0.004430,0.060908,0.024363,0.008859,0.017719,0.026578,0.045404,...,0.004560,0.003075,0.003287,0.002121,0.001485,0.001166,0.001591,0.000318,0.000636,0
3,0.035749,0.013527,0.000000,0.009662,0.052174,0.021256,0.010628,0.016425,0.029952,0.052174,...,0.004812,0.003609,0.003867,0.003007,0.002320,0.000945,0.001117,0.000773,0.000430,0
4,0.031008,0.021041,0.000000,0.001107,0.016611,0.057586,0.014396,0.011074,0.017719,0.013289,...,0.006787,0.004242,0.002651,0.002863,0.002757,0.000530,0.001060,0.000848,0.000318,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484233,0.032115,0.019934,0.000000,0.000000,0.007752,0.050941,0.022148,0.014396,0.009967,0.009967,...,0.004666,0.003499,0.002439,0.002227,0.002121,0.001273,0.000954,0.000636,0.000212,0
484234,0.039867,0.012182,0.000000,0.007752,0.056478,0.024363,0.012182,0.014396,0.019934,0.044297,...,0.004242,0.004136,0.003499,0.001485,0.001485,0.001166,0.001166,0.000848,0.000318,0
484235,0.037681,0.011594,0.000000,0.008696,0.053140,0.018357,0.013527,0.014493,0.023188,0.048309,...,0.004468,0.004554,0.002836,0.002148,0.001890,0.001719,0.000773,0.000773,0.000430,0
484236,0.038647,0.010628,0.000000,0.010628,0.054106,0.017391,0.016425,0.013527,0.038647,0.043478,...,0.005671,0.005070,0.003351,0.002750,0.002664,0.001547,0.001117,0.001203,0.000687,0
